**Instalar/actualizar dependencias en el kernel del notebook**


In [1]:
!pip install -U tensorflow keras

In [2]:
!pip install matplotlib


**Imports y rutas**

In [3]:
from pathlib import Path
import os, json, math, random
import numpy as np
import tensorflow as tf
import keras
from keras import layers

# Rutas (el notebook está en /mansory)
ROOT = Path.cwd()
TRAIN_DIR = ROOT / "train"
TEST_DIR  = ROOT / "test"

IMG_SIZE = (224, 224)
BATCH    = 32
SEED     = 42
AUTOTUNE = tf.data.AUTOTUNE

# Extensiones válidas
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}


**Escaneo de carpetas, clases y pesos**

In [4]:
def gather_images(root: Path):
    paths, labels = [], []
    for top in sorted([p for p in root.iterdir() if p.is_dir()], key=lambda p: p.name.lower()):
        name = top.name
        if name.lower().strip() == "sin grietas":
            cname = "sin_grietas"
            for f in top.rglob("*"):
                if f.suffix.lower() in IMG_EXTS:
                    paths.append(str(f))
                    labels.append(cname)
        else:
            for sev in ["Grave", "Leve", "Moderada"]:
                sub = top / sev
                if not sub.exists(): 
                    continue
                cname = f"{name}_{sev}"   # p.ej. "compresion_vertical_Grave"
                for f in sub.rglob("*"):
                    if f.suffix.lower() in IMG_EXTS:
                        paths.append(str(f))
                        labels.append(cname)
    return paths, labels

train_paths, train_labels = gather_images(TRAIN_DIR)
test_paths,  test_labels  = gather_images(TEST_DIR)

# Lista de clases (orden alfabético)
class_names = sorted(sorted(set(train_labels)), key=lambda s: s)
class_to_idx = {c:i for i,c in enumerate(class_names)}
idx_to_class = {i:c for c,i in class_to_idx.items()}

print("Clases (", len(class_names), "):")
for i,c in enumerate(class_names):
    print(f"{i}: {c}")

# Conteos y pesos de clase
from collections import Counter
counts = Counter(train_labels)
total  = sum(counts.values())
n_cls  = len(class_names)
class_weight = {class_to_idx[c]: total/(n_cls*counts[c]) for c in class_names}
print("\nImágenes por clase (train):")
for c in class_names:
    print(f"{c} -> {counts[c]}")
print("\nclass_weight:", class_weight)


Clases ( 10 ):
0: compresion_vertical_Grave
1: compresion_vertical_Leve
2: compresion_vertical_Moderada
3: friccion cortante_escalonada_Grave
4: friccion cortante_escalonada_Leve
5: friccion cortante_escalonada_Moderada
6: sin_grietas
7: tension diagonal_inclinadas_Grave
8: tension diagonal_inclinadas_Leve
9: tension diagonal_inclinadas_Moderada

Imágenes por clase (train):
compresion_vertical_Grave -> 279
compresion_vertical_Leve -> 22
compresion_vertical_Moderada -> 688
friccion cortante_escalonada_Grave -> 330
friccion cortante_escalonada_Leve -> 28
friccion cortante_escalonada_Moderada -> 1959
sin_grietas -> 23553
tension diagonal_inclinadas_Grave -> 18
tension diagonal_inclinadas_Leve -> 3
tension diagonal_inclinadas_Moderada -> 231

class_weight: {0: 9.717204301075268, 1: 123.23181818181818, 2: 3.9405523255813955, 3: 8.215454545454545, 4: 96.825, 5: 1.3839203675344565, 6: 0.11510635587823208, 7: 150.61666666666667, 8: 903.7, 9: 11.736363636363636}


**tf.data + augmentación conservadora**

In [5]:
def decode_resize(path):
    img = tf.io.read_file(path)
    img = tf.io.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.image.convert_image_dtype(img, tf.float32)   # [0,1]
    return img

data_aug = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),      # ±5° (evita girar compresión vertical a diagonal)
    layers.RandomZoom(0.10),
    layers.RandomTranslation(0.05, 0.05),
    layers.RandomContrast(0.15),
], name="data_augmentation")

def make_dataset(paths, labels, training=False):
    y = [class_to_idx[l] for l in labels]
    ds_x = tf.data.Dataset.from_tensor_slices(paths)
    ds_y = tf.data.Dataset.from_tensor_slices(tf.one_hot(y, depth=len(class_names)))
    ds = tf.data.Dataset.zip((ds_x, ds_y))
    ds = ds.shuffle(len(paths), seed=SEED) if training else ds
    ds = ds.map(lambda p, y: (decode_resize(p), y), num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.map(lambda x, y: (data_aug(x, training=True), y), num_parallel_calls=AUTOTUNE)
    return ds.batch(BATCH).prefetch(AUTOTUNE)

train_ds = make_dataset(train_paths, train_labels, training=True)
val_ds   = make_dataset(test_paths,  test_labels,  training=False)


**Modelo (cabeza) + callbacks**

In [ ]:
inputs = layers.Input((*IMG_SIZE, 3))
base = keras.applications.MobileNetV2(include_top=False, weights="imagenet", input_shape=(*IMG_SIZE,3))
base.trainable = False

x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="categorical_crossentropy",
              metrics=["accuracy", keras.metrics.TopKCategoricalAccuracy(k=3, name="top3")])

class SaveOn95(keras.callbacks.Callback):
    def __init__(self, path="modelo_mansory_95.keras", monitor="val_accuracy", threshold=0.95):
        super().__init__()
        self.path, self.monitor, self.threshold = path, monitor, threshold
        self.saved = False
    def on_epoch_end(self, epoch, logs=None):
        val = logs.get(self.monitor)
        if val is not None and val >= self.threshold and not self.saved:
            self.model.save(self.path)
            self.saved = True
            print(f"\n✅ Guardado {self.path} ( {self.monitor}={val:.3f} )")

cbs = [
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor="val_accuracy"),
    keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3, monitor="val_accuracy"),
    keras.callbacks.ModelCheckpoint("modelo_mansory.keras", save_best_only=True, monitor="val_accuracy"),
    SaveOn95("modelo_mansory_95.keras", threshold=0.95),
]

hist = model.fit(train_ds, validation_data=val_ds, epochs=15,
                 class_weight=class_weight, callbacks=cbs)

# Guarda también las clases
with open("clases_mansory.json","w",encoding="utf-8") as f:
    json.dump({"class_names": class_names}, f, ensure_ascii=False, indent=2)


Epoch 1/15
 77/848 ━━━━━━━━━━━━━━━━━━━━ 11:01 857ms/step - accuracy: 0.4319 - loss: 2.0504 - top3: 0.7303